In [1]:
!pip install torch_geometric


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
"""from google.colab import drive
drive.mount('/content/drive')"""

"from google.colab import drive\ndrive.mount('/content/drive')"

In [3]:
# Imports
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
# PyG
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# Vostri moduli
from src.loadData    import (GraphDataset, TestDataset)
from src.models      import (GNN, GCODWrapper)
from src.transforms import EdgeDropout, NodeDropout, Compose
from src.losses import (
    GCODLoss,
    NoisyCrossEntropyLoss,
    GeneralizedCELoss,
    SymmetricCELoss,
    estimate_transition_matrix,
    ForwardCorrectionLoss,
    BootstrappingLoss
)
from src.divide_mix_def import DivideMixTrainer
from src.utils import set_seed



In [4]:
def add_zeros(data):
    if not hasattr(data, 'x') or data.x is None:
        data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [5]:
"""def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch):
    
    model = model.to(device)
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    print(f"Train loss/acc: {total_loss / len(data_loader):.4f}/{correct / total:.4f}")
    return total_loss / len(data_loader),  correct / total"""

'def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch):\n    \n    model = model.to(device)\n    model.train()\n    total_loss = 0\n    correct = 0\n    total = 0\n    \n    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):\n        data = data.to(device)\n        optimizer.zero_grad()\n        output = model(data)\n        loss = criterion(output, data.y)\n        loss.backward()\n        optimizer.step()\n        total_loss += loss.item()\n        pred = output.argmax(dim=1)\n        correct += (pred == data.y).sum().item()\n        total += data.y.size(0)\n\n    # Save checkpoints if required\n    if save_checkpoints:\n        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"\n        torch.save(model.state_dict(), checkpoint_file)\n        print(f"Checkpoint saved at {checkpoint_file}")\n\n    print(f"Train loss/acc: {total_loss / len(data_loader):.4f}/{correct / total:.4f}")\n  

In [6]:
def validate(model, val_loader, device):
    """Validate model performance on clean validation set."""
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for batch in val_loader:
            batch = batch.to(device)
            outputs = model(batch)
            preds = outputs.argmax(1)
            y_pred.extend(preds.cpu().tolist())
            y_true.extend(batch.y.cpu().tolist())
    
    acc = sum(p == t for p, t in zip(y_pred, y_true)) / len(y_true)
    f1 = f1_score(y_true, y_pred, average='macro')
    return acc, f1

def compute_centroids(model, dataloader, device):
    model.eval()
    all_embeddings = []
    all_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            batch = batch.to(device)
            _ = model(batch)
            all_embeddings.append(model.embeddings)
            all_labels.append(batch.y)
    
    embeddings = torch.cat(all_embeddings)
    labels = torch.cat(all_labels)

    centroids = {}
    for label in torch.unique(labels):
        label_mask = labels == label
        centroids[int(label.item())] = embeddings[label_mask].mean(dim=0)
    
    return centroids


def train(
    model,
    train_dataset,
    val_dataset,
    criterion,
    main_optimizer,
    ub_optimizer, 
    device,
    num_epochs=100,
    patience=10,
    batch_size=32,
):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    best_val_f1 = 0
    patience_counter = 0
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        y_true_train, y_pred_train = [], []
        model = model.to(device)
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            batch = batch.to(device)
            labels = batch.y
            
            torch.autograd.set_detect_anomaly(True)
            logits = model(batch)
            current_uB = model.uB[batch.idx]

            with torch.no_grad():
                preds = torch.argmax(logits, dim=1)      # predicted class indices
                correct = (preds == labels).sum().item() # number of correct predictions
                total = labels.size(0)                   # batch size
                train_acc = correct / total              # batch accuracy

            
            loss, l1, l2, l3 = criterion(logits=logits, 
                                         uB=current_uB,
                                         labels=labels, 
                                         train_acc=train_acc)


            

            main_optimizer.zero_grad()
            (l1+l3).backward(retain_graph=True)
            main_optimizer.step()

            # --- uB update pass ---
            with torch.no_grad():
                logits = model.gnn(batch)  # Recompute with updated θ
            
            current_uB = model.uB[batch.idx].requires_grad_()
            probs = F.softmax(logits, dim=1)
            num_classes = 6
            yB = F.one_hot(labels, num_classes).float()
            adjusted_probs = probs + torch.diag(current_uB) @ yB
            L2 = torch.norm(adjusted_probs - yB, p=2)**2 / num_classes
            
            ub_optimizer.zero_grad()
            L2.backward()
            ub_optimizer.step()
            
            with torch.no_grad():
                model.uB.clamp_(0, 1)

            total_loss += loss.item()
            preds = logits.argmax(dim=1)

            y_pred_train.extend(preds.cpu().tolist())
            y_true_train.extend(labels.cpu().tolist())

        # Compute training metrics
        train_acc = sum(p == t for p, t in zip(y_pred_train, y_true_train)) / len(y_true_train)
        train_f1 = f1_score(y_true_train, y_pred_train, average='macro')
        avg_train_loss = total_loss / len(train_loader)

        # Validation
        val_acc, val_f1 = validate(model, val_loader, device)

        print(f"Epoch {epoch+1}: "
              f"Train Loss={avg_train_loss:.4f}, "
              f"Train Acc={train_acc:.4f}, "
              f"Train F1={train_f1:.4f}, "
              f"Val Acc={val_acc:.4f}, "
              f"Val F1={val_f1:.4f} (Best={best_val_f1:.4f})")

        # Early stopping based on F1
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_state = deepcopy(model.state_dict())
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered!")
                break

    # Restore best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    return model


In [7]:
def warmup_train(
    model,
    train_dataset,
    val_dataset,
    device,
    num_epochs=20,
    patience=5,
    batch_size=32,
    lr=1e-3
):
    model = model.to(device)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    train_losses, val_losses = [], []
    train_f1s, val_f1s = [], []

    best_val_f1 = 0
    patience_counter = 0
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        y_true_train, y_pred_train = [], []

        for batch in tqdm(train_loader, desc=f"[Warmup] Epoch {epoch+1}/{num_epochs}"):
            batch = batch.to(device)
            labels = batch.y

            logits = model.gnn(batch)  # Only run GNN, no wrapper logic needed
            loss = criterion(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            y_true_train.extend(labels.cpu().tolist())
            y_pred_train.extend(preds.cpu().tolist())

        # Training metrics
        train_acc = sum(p == t for p, t in zip(y_pred_train, y_true_train)) / len(y_true_train)
        train_f1 = f1_score(y_true_train, y_pred_train, average='macro')
        avg_train_loss = epoch_loss / len(train_loader)

        train_losses.append(avg_train_loss)
        train_f1s.append(train_f1)

        # Validation
        val_acc, val_f1 = validate(model, val_loader, device)
        val_f1s.append(val_f1)

        print(f"[Warmup] Epoch {epoch+1}: "
              f"Train Loss={avg_train_loss:.4f}, Train Acc={train_acc:.4f}, Train F1={train_f1:.4f}, "
              f"Val Acc={val_acc:.4f}, Val F1={val_f1:.4f} (Best={best_val_f1:.4f})")

        # Early stopping
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_state = deepcopy(model.state_dict())
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("[Warmup] Early stopping triggered!")
                break

    if best_model_state:
        model.load_state_dict(best_model_state)

    return model

In [8]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)

            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())

    
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
        print(f"Test loss/acc {total_loss / len(data_loader):.4f} / {correct / total:.4f}")
    return predictions

In [9]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd()
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))

    os.makedirs(submission_folder, exist_ok=True)

    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")

    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })

    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [10]:
def plot_training_progress(train_losses, train_accuracies, save_plot, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    if(save_plot):
        # Save plots in the current directory
        os.makedirs(output_dir, exist_ok=True)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "training_progress.png"))
        plt.close()

In [11]:
# Hyper-parameters
device                =  "cuda" if torch.cuda.is_available() else "cpu"
# Modello
gnn_type              = 'gin-virtual'   
num_layer             = 3
emb_dim               = 300
drop_ratio            = 0.5

pooling               = "mean"

edge_p  = 0.5   # frazione di bordi da droppare
node_p = 0.5

epochs                = 100
weight_decay          = 1e-4
num_classes           = 6
batch_size            = 32 
warmup_steps          = 10
patience              = 12
residual              = False
model_lr              = 0.001
u_lr                  = 1

seed                  = set_seed(42)
transforms = Compose([
    add_zeros,
])

# Epoch 49: Train Loss=0.3052, Train Acc=0.9117, Val Acc=0.6953 DATASET C

# Minimum Effort

In [12]:
def create_gnn_model(gnn_type, num_classes, num_layer, emb_dim, drop_ratio, device, residual, pooling):
    kwargs = {
        'gnn_type': gnn_type.replace("-virtual", ""),
        'num_class': num_classes,
        'num_layer': num_layer,
        'emb_dim': emb_dim,
        'drop_ratio': drop_ratio,
        'virtual_node': "virtual" in gnn_type,
        'residual': residual,
        'graph_pooling': pooling
        }

    model = GNN(**kwargs).to(device)
    return model

In [13]:
def split_dataset(dataset: GraphDataset, val_ratio=0.1, seed=42):
    labels = torch.tensor([data.y.item() for data in dataset])
    indices = list(range(len(dataset)))

    train_idx, val_idx = train_test_split(
        indices,
        test_size=val_ratio,
        stratify=labels,
        random_state=seed
    )

    train_subset = []
    val_subset = []

    for new_idx, original_idx in enumerate(train_idx):
        data = dataset[original_idx]
        data.idx = new_idx  # Normalize index
        train_subset.append(data)

    for new_idx, original_idx in enumerate(val_idx):
        data = dataset[original_idx]
        data.idx = new_idx  # Normalize index
        val_subset.append(data)

    return train_subset, val_subset


In [ ]:
import gc 
# Modifica loop principale di training
train_datasets_path = ["datasets/C/train.json.gz"]

for ds in train_datasets_path:


    print("Generating dataset")
    
    full_dataset = GraphDataset(ds, transform=transforms).shuffle()
    #full_dataset = TestDataset()
    print("Splitting dataset")
    train_set, val_set = split_dataset(full_dataset, 0.2, seed)
    

    model_kwargs = {"gnn_type": gnn_type, 
                    "num_classes":num_classes, 
                    "num_layer": num_layer, 
                    "emb_dim": emb_dim, 
                    "drop_ratio": drop_ratio, 
                    "device":device, 
                    "residual": residual, 
                    "pooling": pooling,
                    }
    
    model = create_gnn_model(**model_kwargs)

    wrapper = GCODWrapper(model, len(train_set))
    criterion = GCODLoss(num_classes=num_classes, batch_size=batch_size, device=device)
    
    theta_optim = torch.optim.Adam(wrapper.gnn.parameters(), lr=model_lr, weight_decay=weight_decay)
    u_optim = torch.optim.Adam([wrapper.uB], lr=u_lr, weight_decay=weight_decay)

    warmup_criterion = nn.CrossEntropyLoss()
    
    print(f"Warmup Step ({warmup_steps} Epochs)")
    warmup_model = warmup_train(model=wrapper, 
                       train_dataset=train_set,
                       val_dataset=val_set,
                       device=device,
                       num_epochs=warmup_steps,
                       patience = warmup_steps+1,
                       batch_size=batch_size 
                       )

    print(f"Train ({epochs} Epochs)")
    best_model = train(model=warmup_model, 
                       train_dataset=train_set, 
                       val_dataset=val_set, 
                       criterion=criterion, 
                       main_optimizer=theta_optim,
                       ub_optimizer = u_optim,  
                       device=device,
                       num_epochs=epochs, 
                       patience=patience, 
                       batch_size=batch_size)

    del val_set, train_set, full_dataset, criterion
    gc.collect()

    # Test e predizioni
    test_loader = DataLoader(GraphDataset(ds.replace("train", "test"), transform=transforms), batch_size=32)
    #test_loader = DataLoader(TestDataset(), batch_size=batch_size)
    predictions = evaluate(test_loader, best_model, device, False)
    save_predictions(predictions=predictions, test_path=ds.replace("train", "test"))
    del test_loader, best_model, predictions
    gc.collect()
    torch.cuda.empty_cache()

Generating dataset
Splitting dataset
Warmup Step (10 Epochs)


[Warmup] Epoch 1/10: 100%|██████████| 282/282 [00:08<00:00, 32.45it/s]


[Warmup] Epoch 1: Train Loss=1.6957, Train Acc=0.3125, Train F1=0.1762, Val Acc=0.3262, Val F1=0.1501 (Best=0.0000)


[Warmup] Epoch 2/10: 100%|██████████| 282/282 [00:08<00:00, 33.10it/s]


[Warmup] Epoch 2: Train Loss=1.5927, Train Acc=0.3709, Train F1=0.2080, Val Acc=0.3497, Val F1=0.2006 (Best=0.1501)


...

Epoch 78: Train Loss=2.5521, Train Acc=0.7503, Train F1=0.7077, Val Acc=0.6950, Val F1=0.6468 (Best=0.6729)


Epoch 79/100: 100%|██████████| 282/282 [00:18<00:00, 15.37it/s]


Epoch 79: Train Loss=2.5415, Train Acc=0.7512, Train F1=0.7029, Val Acc=0.7004, Val F1=0.6519 (Best=0.6729)
Early stopping triggered!


Iterating eval graphs: 100%|██████████| 74/74 [00:05<00:00, 14.26batch/s]


Predictions saved to /home/flaviolinux/uni/deep_learning/hackaton/submission/testset_A.csv
